In [ ]:
!pip install finance-datareader

In [ ]:
import pandas as pd
import pandas_datareader as web
import math
import os
import numpy as np
import FinanceDataReader as fdr
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tqdm import tqdm

In [ ]:
#  차트 설정
%matplotlib inline
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = 'nanummyeongjo'
plt.rcParams["figure.figsize"] = (14,4)
plt.rcParams['lines.linewidth'] = 2
plt.rcParams["axes.grid"] = True

In [ ]:
%%html
<style>
    table { display: inline-block }
    .rendered_html td, .rendered_html th { text-align: left; }
</style>

In [ ]:
path = './'
list_name = 'Stock_List.csv'
stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
371,더네이쳐홀딩스,298540,KOSDAQ
372,코엔텍,029960,KOSDAQ
373,원익홀딩스,030530,KOSDAQ
374,웹케시,053580,KOSDAQ


In [ ]:
start_date = '20210101'
end_date = '20210808'
sample_code = '005930'
stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,0.024691
2021-01-05,81600,83900,81600,83900,35335669,0.010843
2021-01-06,83300,84500,82100,82200,42089013,-0.020262
2021-01-07,82800,84200,82700,82900,32644642,0.008516
2021-01-08,83300,90000,83000,88800,59013307,0.071170
...,...,...,...,...,...,...
2021-08-02,79200,79500,78700,79300,11739124,0.010191
2021-08-03,79400,81400,79300,81400,24339360,0.026482
2021-08-04,82200,83100,81800,82900,25642368,0.018428


In [ ]:
start_date = '20210101'
end_date = '20210808'
sample_code = '000150'
stock = fdr.DataReader(sample_code, start = '20210809', end = '20210813')
stock

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-08-09,96600,96700,94200,95000,71184,-0.022634
2021-08-10,95300,97000,93900,96300,70570,0.013684
2021-08-11,95400,99400,95400,96700,53718,0.004154
2021-08-12,97700,98800,97000,98100,36278,0.014478
2021-08-13,98700,98700,93000,93400,85748,-0.047910


In [ ]:
data_sample = stock.filter(['Close'])
dataset_sample = data_sample.values

In [ ]:
sample_training_data_len = math.ceil(len(dataset_sample)*.8)

sample_training_data_len

320

In [ ]:
#Scale the data
scaler = MinMaxScaler(feature_range=(0,1))
sample_scaled_data = scaler.fit_transform(dataset_sample)


In [ ]:
sample_train_data = sample_scaled_data[0:sample_training_data_len,:]
sample_x_train = []
sample_y_train = []

for i in range(60, len(sample_train_data)):
  sample_x_train.append(sample_train_data[i-60:i,0])
  sample_y_train.append(sample_train_data[i,0])
  if i<=61:
    print(sample_x_train)
    print(sample_y_train)
    print()

[array([0.26185567, 0.26804124, 0.26804124, 0.2742268 , 0.29484536,
       0.33195876, 0.35051546, 0.36082474, 0.36082474, 0.34020619,
       0.37525773, 0.38762887, 0.41030928, 0.38969072, 0.40824742,
       0.37731959, 0.33608247, 0.34226804, 0.30309278, 0.28659794,
       0.30309278, 0.33814433, 0.35051546, 0.38350515, 0.36907216,
       0.35463918, 0.35876289, 0.37113402, 0.37525773, 0.39793814,
       0.39175258, 0.35670103, 0.36494845, 0.36082474, 0.3443299 ,
       0.29484536, 0.31752577, 0.28865979, 0.27628866, 0.24123711,
       0.25773196, 0.26597938, 0.30721649, 0.31546392, 0.28865979,
       0.24123711, 0.24948454, 0.19793814, 0.17113402, 0.15360825,
       0.13195876, 0.09896907, 0.06391753, 0.00927835, 0.05979381,
       0.        , 0.09175258, 0.12680412, 0.10927835, 0.11958763])]
[0.1103092783505154]

[array([0.26185567, 0.26804124, 0.26804124, 0.2742268 , 0.29484536,
       0.33195876, 0.35051546, 0.36082474, 0.36082474, 0.34020619,
       0.37525773, 0.38762887, 0.410

In [ ]:
sample_x_train, sample_y_train = np.array(sample_x_train), np.array(sample_y_train)

In [ ]:
sample_x_train = np.reshape(sample_x_train, (sample_x_train.shape[0],sample_x_train.shape[1],1))

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(sample_x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

NameError: ignored

In [ ]:
#compile
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#train
model.fit(sample_x_train,sample_y_train, batch_size=1, epochs=1)

260/260 [==============================] - 10s 28ms/step - loss: 0.0101


In [ ]:
#Create the testing data set
#Create a new array containing scaled values from index 1543 to 2003
test_data = sample_scaled_data[sample_training_data_len-60:,:]
sample_x_test=[]
sample_y_test=dataset_sample[sample_training_data_len:, :]

In [ ]:
data_sample = stock.filter(['Close'])
dataset_sample = data_sample.values

In [ ]:
code='000080'

In [ ]:
sample_name = 'sample_submission.csv'
sample_submission = pd.read_csv(os.path.join(path,sample_name))

In [ ]:
data = fdr.DataReader(code, start = 20210101, end = end_date).filter(['Close'])

In [ ]:
data

,Close
Date,
1997-07-29,2095
1997-07-30,2027
1997-07-31,2016
1997-08-01,2027
1997-08-02,2036
...,...
2021-08-02,24400
2021-08-03,24500
2021-08-04,25600


In [ ]:
data = fdr.DataReader(code, start = '20100101', end = end_date).filter(['Close'])
dataset = data.values

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

train_data = scaled_data[0:len(dataset),:]
x_train = []
y_train = []

for i in range(77, len(train_data)):
  x_train.append(train_data[i-77:i,0])
  y_train.append(train_data[i,0])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train,y_train, batch_size=3, epochs=2)


test_data = fdr.DataReader(code, start = '20210406', end = end_date).filter(['Close'])
test_dataset = test_data.values

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(test_dataset)

test_data = scaled_data[0:len(test_dataset),:]
x_test = []

for i in range(77, len(test_data)):
  x_test.append(test_data[i-77:i,0])
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

predicted = model.predict(x_test)
predicted = scaler.inverse_transform(predicted)

sample_submission.loc[:,code] = predicted 


Epoch 1/2
929/929 [==============================] - 40s 40ms/step - loss: 0.0025
Epoch 2/2
929/929 [==============================] - 37s 40ms/step - loss: 0.0010


In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130,000020
0,2021-08-09,18748.191406,35101.679688,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
1,2021-08-10,18980.589844,34942.031250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
2,2021-08-11,19225.320312,34780.042969,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
3,2021-08-12,19452.267578,34628.105469,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
4,2021-08-13,19665.691406,34520.585938,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
5,2021-08-30,19851.939453,34445.003906,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
6,2021-08-31,20035.931641,34312.722656,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
7,2021-09-01,20153.517578,34133.863281,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
8,2021-09-02,20234.992188,34070.027344,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
9,2021-09-03,20309.806641,34130.476562,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [ ]:
len(test_data)

87

In [ ]:
df_predicted=pd.DataFrame(predicted)
df_predicted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       27 non-null     float32
dtypes: float32(1)
memory usage: 236.0 bytes


In [ ]:
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = '20190101', end = end_date).filter(['Close'])
    dataset = data.values

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(dataset)

    train_data = scaled_data[0:len(dataset),:]
    x_train = []
    y_train = []

    for i in range(77, len(train_data)):
      x_train.append(train_data[i-77:i,0])
      y_train.append(train_data[i,0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1],1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(x_train,y_train, batch_size=1, epochs=1)


    test_data = fdr.DataReader(code, start = '20210406', end = end_date).filter(['Close'])
    test_dataset = test_data.values

    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(test_dataset)

    test_data = scaled_data[0:len(test_dataset),:]
    x_test = []

    for i in range(77, len(test_data)):
      x_test.append(test_data[i-77:i,0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    predicted = model.predict(x_test)
    predicted = scaler.inverse_transform(predicted)

    sample_submission.loc[:,code] = predicted 

sample_submission.isna().sum().sum()


  0%|          | 0/376 [00:00<?, ?it/s]

567/567 [==============================] - 25s 38ms/step - loss: 0.0042


  0%|          | 1/376 [00:29<3:02:13, 29.16s/it]

567/567 [==============================] - 24s 38ms/step - loss: 0.0085


  1%|          | 2/376 [00:57<2:58:10, 28.59s/it]

...

550/550 [==============================] - 25s 39ms/step - loss: 0.0065


100%|█████████▉| 375/376 [3:14:19<00:32, 32.00s/it]

567/567 [==============================] - 24s 38ms/step - loss: 0.0066


100%|██████████| 376/376 [3:14:47<00:00, 31.08s/it]


10

In [ ]:
predicted

NameError: ignored

In [ ]:
sample_submission.to_csv('BASELINE_Linear.csv',index=False)

In [ ]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,...,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,298540,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130,000020
0,2021-08-09,21769.189453,35207.996094,61628.367188,179946.750000,115402.859375,18570.853516,52707.390625,87135.609375,118957.335938,673617.0625,56368.312500,217534.734375,30382.140625,64943.519531,20480.722656,2609.098877,25156.560547,6085.913574,305956.43750,61770.035156,20765.500000,35415.128906,9848.725586,8036.996094,30775.724609,158319.109375,48538.503906,71237.585938,51943.679688,275929.09375,323901.46875,92334.015625,117814.226562,38672.855469,33552.261719,28944.558594,139849.546875,226264.046875,112207.750000,...,233036.906250,14153.069336,168541.468750,251270.250000,6792.691406,8685.855469,33622.804688,14940.499023,82416.804688,67180.523438,61006.828125,31389.242188,406964.75000,880612.5000,594894.0000,20690.218750,33476.578125,69133.203125,132663.906250,17868.373047,129461.257812,38785.609375,11354.521484,5069.663574,42043.750000,23122.130859,86916.523438,125930.523438,5884.028809,46470.324219,50142.300781,58529.050781,47942.035156,32217.162109,64452.128906,301922.37500,332395.78125,31121.041016,27460.978516,NaN
1,2021-08-10,21972.640625,35134.656250,61533.179688,179838.031250,114129.054688,18498.162109,52509.101562,86988.179688,118561.781250,679678.7500,55839.738281,217290.359375,30339.523438,64990.050781,20520.931641,2583.129150,25144.470703,6093.976074,307817.56250,61563.144531,20727.818359,35441.546875,9804.652344,8028.366699,30690.417969,157796.453125,48701.878906,71496.539062,51925.218750,274998.43750,323455.53125,92061.171875,118153.484375,38555.144531,33445.769531,29118.076172,139451.812500,225706.203125,111825.968750,...,232992.296875,14135.000000,168147.578125,250896.671875,6811.520020,8652.782227,33659.703125,14920.958008,83939.281250,68465.679688,61143.375000,31240.984375,407823.21875,878550.5625,603438.9375,20627.019531,33316.953125,69117.593750,131093.750000,18089.587891,129019.953125,38845.183594,11319.886719,5045.741699,42183.050781,23112.044922,86710.296875,124993.968750,5883.629395,46412.769531,50215.582031,58649.003906,49408.714844,32199.648438,64348.964844,301447.28125,334922.59375,31258.187500,27115.597656,NaN
2,2021-08-11,22202.455078,35053.261719,61483.882812,179823.171875,112835.500000,18431.322266,52318.757812,86857.968750,118118.453125,683933.3750,55346.515625,216941.640625,30301.335938,64895.531250,20610.015625,2565.627930,25126.267578,6101.158203,309153.34375,61344.273438,20688.253906,35442.652344,9785.865234,8020.478516,30637.474609,157068.859375,48852.269531,71698.968750,52027.953125,274179.87500,322827.62500,91790.843750,118294.734375,38417.929688,33299.453125,29225.087891,138829.093750,225178.703125,111396.343750,...,232535.062500,14117.951172,167445.500000,250388.218750,6838.418945,8624.471680,33724.871094,14904.386719,85250.382812,69478.000000,61276.406250,31114.978516,408454.50000,877101.7500,609967.7500,20593.197266,33180.238281,69135.226562,129572.007812,18257.810547,128513.507812,38824.246094,11280.125000,5033.932617,42364.371094,23030.337891,86439.531250,124149.921875,5885.820801,46397.601562,50206.218750,58737.429688,50036.394531,32164.966797,64270.496094,300869.28125,336998.12500,31373.099609,26987.236328,NaN
3,2021-08-12,22475.162109,34966.523438,61467.839844,179666.406250,111578.781250,18365.187500,52111.976562,86725.421875,117572.585938,686811.3125,55000.292969,216652.859375,30268.515625,64601.859375,20722.767578,2551.082520,25166.181641,6108.637207,310836.96875,61200.878906,20641.601562,35506.636719,9772.728516,8016.3212

In [ ]:
sample_name2 = 'sample_submission2.csv'
sample_submission2 = pd.read_csv(os.path.join(path,sample_name))

NameError: ignored